In [1]:
import glob
import re
import pandas as pd
import time

from src.load_data import Instance
from src.heuristic import heuristic
from src.combined_heuristic import combined_heuristic

In [2]:
solution_path = "../data/j60lb.sm"

with open(solution_path) as f:
    lines = f.read()

In [3]:
solutions = {}
separator = re.compile("\=+")
for s in separator.split(lines)[4].split("\n")[1:-1]:
    if s.split()[4]=='*':
        solutions['j60' + s.split()[0] + '_' + s.split()[1] + '.sm'] = int(s.split()[2])

In [4]:
folder_path = "../data/j60.sm/"

results = pd.DataFrame(columns = ["instance", "optimum_makespan", "matheus_makespan", "matheus_time", "belen_makespan", "belen_time", "combined_makespan", "combined_time"])

matheus_results = pd.read_csv("../data/results/matheus.csv", sep=";")

for sol in solutions:
    problem_path = folder_path + sol
    instance = Instance.read_instance(problem_path)
    start = time.time()
    solution_belen =  heuristic(instance)
    end = time.time()
    belen_time = end - start
    start = time.time()
    solution_combined =  combined_heuristic(instance)
    end = time.time()
    combined_time = end - start
    
    m = matheus_results[matheus_results["Instance"]==sol]
    
    d = {
        "instance": [sol], 
        "optimum_makespan": [solutions[sol]], 
        "matheus_makespan": [m["Heuristic makespan"].item()], 
        "matheus_time": [m["Execution time (s)"].item()], 
        "belen_makespan": [solution_belen.makespan], 
        "belen_time": [belen_time],
        "combined_makespan": [solution_combined.makespan],
        "combined_time": [combined_time]
    }
    df_new_row = pd.DataFrame.from_records(d)
    results = pd.concat([results, df_new_row])

In [5]:
results.to_csv("../data/results/results.csv")